# Error Handling

We show how the optimizer in Trace can be used to resolve execution error duing optimization. When an exception is thrown during the execution of a bundled method, a special MessageNode, called the ExceptionNode, is created and a new Python exception trace.ExecutionError is thrown. The trace.ExecutionError is a wrapper of the original exception and contains the created ExceptionNode as its attribute `exception_node`. The ExceptionNode's parents are the inputs to the bundled method triggering the exception. Therefore, to resolve the error, we can simply use the created ExceptionNode as the target and its data can be used as feedback. 


Below we show a basic example of how Trace deal with execptions.

In [ ]:
%pip install trace-opt

In [2]:
from opto import trace
from opto.optimizers import OptoPrime


def check_input(a):
    if a <0.1:
        raise ValueError("Input must be greater than 0.1")

@trace.bundle()
def func_with_input_checking(a):
    check_input(a)
    return True


param = trace.node(-1., trainable=True)  # Note; setting the initial value to -1. makes it a float;
optimizer = OptoPrime([param], memory_size=5)

for _ in range(5):
    try:
        success = func_with_input_checking(param)
        print(f'\nSuccess, Parameter: {param.data}')
        break
    except trace.ExecutionError as e:
        print(f'\nIter {_}, Failed, Parameter {param.data}\n')
        target = e.exception_node
        optimizer.zero_feedback()
        optimizer.backward(target, target.create_feedback())
        optimizer.step(verbose=True)



Iter 0, Failed, Parameter -1.0

Prompt
 
You're tasked to solve a coding/algorithm problem. You will see the instruction, the code, the documentation of each function used in the code, and the feedback about the execution result.

Specifically, a problem will be composed of the following parts:
- #Instruction: the instruction which describes the things you need to do or the question you should answer.
- #Code: the code defined in the problem.
- #Documentation: the documentation of each function used in #Code. The explanation might be incomplete and just contain high-level description. You can use the values in #Others to help infer how those functions work.
- #Variables: the input variables that you can change.
- #Constraints: the constraints or descriptions of the variables in #Variables.
- #Inputs: the values of other inputs to the code, which are not changeable.
- #Others: the intermediate values created through the code execution.
- #Outputs: the result of the code output.
- #Feed

Next we extend this basic example to create an example of constrained optimization. This example shows how optimization and constrained satisfication can be approached in the same way.

In [3]:
from opto import trace
from opto.optimizers import OptoPrime

trace.GRAPH.clear()

def check_input(a):
    if a <0.1:
        raise ValueError("Input must be greater than 0.1")

@trace.bundle()
def objective(a):
    """ Computes (a+1)**2. """
    check_input(a)
    return (a+1)**2


param = trace.node(-1., trainable=True)  # Note; setting the initial value to -1. makes it a float;
optimizer = OptoPrime([param], memory_size=5)

for _ in range(10):
    try:
        target = objective(param)
        feedback = 'Minimize the objective.'
        print(f'\nIter {_}, Objective {target.data}, Parameter {param.data}\n')

    except trace.ExecutionError as e:
        print(f'\nIter {_}, Not satisfying constraint, Parameter {param.data}\n')
        target = e.exception_node
        feedback = e.exception_node.create_feedback()

    optimizer.zero_feedback()
    optimizer.backward(target, feedback)
    optimizer.step()



Iter 0, Not satisfying constraint, Parameter -1.0


Iter 1, Objective 1.44, Parameter 0.2


Iter 2, Objective 1.2100000000000002, Parameter 0.1


Iter 3, Objective 1.2100000000000002, Parameter 0.1


Iter 4, Objective 1.2321000000000002, Parameter 0.11


Iter 5, Objective 1.2321000000000002, Parameter 0.11


Iter 6, Objective 1.2122009999999999, Parameter 0.101


Iter 7, Not satisfying constraint, Parameter -1.0


Iter 8, Objective 1.2321000000000002, Parameter 0.11


Iter 9, Objective 1.2544000000000002, Parameter 0.12

